In [1]:

import requests
import json
from bs4 import BeautifulSoup

originUrl = 'https://www.behindthename.com'
nameData = []

def PageByPageNum():
    url = 'https://www.behindthename.com/names/usage/english'

    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        names = soup.select('span.listname > a')
        for n in names:
            nameData.append({
                'name': n.get_text(),
                'link':n['href']
            })

    for i in range(2, 15):
        response = requests.get(url + '/' + str(i))
        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            names = soup.select('span.listname > a')
            for n in names:
                nameData.append({
                    'name': n.get_text(),
                    'link': n['href']
                })

def WriteData():
    fileName = "nameHref.json"
    file = open(fileName, "w")
    json.dump(nameData, file)
    file.close()


# Press the green button in the gutter to run the script.
if __name__ == '__main__':
    PageByPageNum()
    # for i in Alpahbet:
    #     print(i)
    #     PageByAlpahbet(i)

    WriteData()
    print(nameData)
    print(len(nameData))

# See PyCharm help at https://www.jetbrains.com/help/pycharm/


[{'name': 'Aaliyah', 'link': '/name/aaliyah'}, {'name': 'Aaren', 'link': '/name/aaren'}, {'name': 'Aaron', 'link': '/name/aaron'}, {'name': 'Abbey', 'link': '/name/abbey'}, {'name': 'Abbi', 'link': '/name/abbi'}, {'name': 'Abbie', 'link': '/name/abbie'}, {'name': 'Abby', 'link': '/name/abby'}, {'name': 'Abe 1', 'link': '/name/abe-1'}, {'name': 'Abegail', 'link': '/name/abegail'}, {'name': 'Abel', 'link': '/name/abel'}, {'name': 'Abi', 'link': '/name/abi'}, {'name': 'Abigail', 'link': '/name/abigail'}, {'name': 'Abigayle', 'link': '/name/abigayle'}, {'name': 'Abilene', 'link': '/name/abilene'}, {'name': 'Abner', 'link': '/name/abner'}, {'name': 'Abraham', 'link': '/name/abraham'}, {'name': 'Abram 1', 'link': '/name/abram-1'}, {'name': 'Acacia', 'link': '/name/acacia'}, {'name': 'Ace 1', 'link': '/name/ace-1'}, {'name': 'Ada 1', 'link': '/name/ada-1'}, {'name': 'Adair', 'link': '/name/adair'}, {'name': 'Adaline', 'link': '/name/adaline'}, {'name': 'Adalyn', 'link': '/name/adalyn'}, {'nam

In [24]:
import requests
import json
from bs4 import BeautifulSoup

originUrl = 'https://www.behindthename.com'
data =[]

def GetGendernUsage(url):
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        # Gender
        gender = 'n'
        male = soup.select_one('span.infoname-info > span.masc')
        female = soup.select_one('span.infoname-info > span.fem')
        if str(male) != 'None' and str(female) != 'None':
            gender='U'
        elif str(female) != 'None':
            gender='F'
        elif str(male) != 'None':
            gender='M'

        usage = []
        usages = soup.select('a.usg')
        for u in usages:
            usage.append(u.get_text())
            
        text = soup.select_one('div.namedef')
        meaning = text.get_text()
        return gender, usage,meaning
    else:
        print(response.status_code)


def GetNameRating(url):  # 이름 하나당 레이팅 값
    # url='https://www.behindthename.com/name/quinti10n/rating'
    # url='https://www.behindthename.com/name/adam/rating'
    response = requests.get(url)

    attribute = {
        "A Good Name": 0, "A Bad Name": 0,
        "Masculine": 0, "Feminine": 0,
        "Classic": 0, "Modern": 0,
        "Mature": 0, "Youthful": 0,
        "Formal": 0, "Informal": 0,
        "Upper Class": 0, "Common": 0,
        "Urban": 0, "Natural": 0,
        "Wholesome": 0, "Devious": 0,
        "Strong": 0, "Delicate": 0,
        "Refined": 0, "Rough": 0,
        "Strange": 0, "Boring": 0,
        "Simple": 0, "Complex": 0,
        "Serious": 0, "Comedic": 0,
        "Nerdy": 0, "Unintellectual": 0
    }

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.select_one('center > table')
        

        if str(table) != 'None':  # Rating 데이터 없음
            humanCount = soup.select_one('center > p')

            slicing = humanCount.get_text()[26:-7]

            cnt = slicing.replace(',', "")
            cnt = int(cnt)
        
            tableRow = table.select('tr')
            index = 0
            for tr in tableRow:
                left = tr.select_one('td:first-child > span').get_text()
                leftPoint = tr.select_one('td:nth-child(2n) > span').get_text()
                right = tr.select_one('td:last-child > span').get_text()
                rightPoint = tr.select_one('td:nth-child(4n) > span').get_text()

                leftPoint = int(leftPoint[:-1])
                rightPoint = int(rightPoint[:-1])
                leftHuman = round(cnt * leftPoint * 0.01)
                rightHuman = round(cnt * rightPoint * 0.01)

                attribute[left]=leftHuman
                attribute[right] = rightHuman

    else:
        print(response.status_code)

    return attribute


def PageByPageNum():
    with open('nameHref.json', 'r') as f:
        nameSet = json.load(f)

    for set in nameSet:
        url = originUrl+set['link']
        gender, usage,meaning = GetGendernUsage(url)
        attribute = GetNameRating(url+'/rating')
        data.append({
            'name': set['name'],
            'gender': gender,
            'usage': usage,
            'meaning': meaning,
            'attribute': attribute
        })
        print(set['name'])


def WriteData():
    fileName = "mvpNameSet_ver1.json"
    file = open(fileName, "w")
    json.dump(data, file)
    file.close()

# Press the green button in the gutter to run the script.

if __name__ == '__main__':
    PageByPageNum()

    WriteData()

# See PyCharm help at https://www.jetbrains.com/help/pycharm/


Aaliyah
Aaren
Aaron
Abbey
Abbi
Abbie
Abby
Abe 1
Abegail
Abel
Abi
Abigail
Abigayle
Abilene
Abner
Abraham
Abram 1
Acacia
Ace 1
Ada 1
Adair
Adaline
Adalyn
Adalynn
Adam
Adamina
Addie
Addilyn
Addison
Addy 1
Addyson
Adela
Adelaide
Adele
Adelia
Adeline
Adella
Adelle
Adelyn
Adelynn
Aden
Adena
Adolph
Adria
Adrian
Adriana
Adrianna
Adrianne
Adrienne
Agatha
Aggie
Agnes
Aidan
Aiden
Aileen
Aimee
Ainslee
Ainsley
Ainslie
Al
Alaia 2
Alaina
Alaiya
Alan
Alana
Alani
Alanis
Alanna
Alannah
Alannis
Alaya
Alayah
Alayna
Alban
Albert
Alberta
Albie
Albin
Alden
Aldous
Alea
Aleah
Alease
Alec
Alecia
Aleesha
Alene
Alesha
Alesia
Aleta
Aletha
Alethea
Alex
Alexa
Alexander
Alexandra
Alexandrea
Alexandria
Alexandrina
Alexia
Alexina
Alexis
Alexus
Alf 2
Alfie
Alfred
Alfreda
Algar
Alger
Algernon
Ali 2
Aliah
Alice
Alicia
Aline
Alise 2
Alisha
Alishia
Alisia
Alison
Alissa
Alisya
Alita
Alivia
Allan
Allana
Allannah
Allegra
Allen
Allie
Allison
Allissa
Ally 1
Allycia
Allyn
Allyson
Alma 1
Alonzo
Aloysius
Alpha
Alphonso
Alphonzo
Alt